# **Avance 6**

## *Análisis fundamental*

Este analisis incluye los siguientes componentes:
* Análisis Top-Down (análisis macroeconomico)
* Análisis Bottom-Up (cálculo de indicadores como PER, Capitalización bursátil, Q de Tobin, Valor Intrínseco, Yield anual, ROA, ROE)
* Benchmark
* Calculo múltiplos 
* Valor por descuento de dividendos

### **Análisis Top-Down**

Se realiza un análisis de los factores que pueden afectar el valor de la compañia, estos se pueden encontrar en noticias o estados financieros y pueden ser a nivel global, del sector o industria o cía.

### **Análisis bottom-up**

En este análisis se utilizan indicadores fundamentales y valoración de acciones para comprender el estado de la acción

In [4]:
pip install --upgrade pandas-datareader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
#importamos lo datos
from pandas_datareader import data as  pdr
from datetime import datetime

start = datetime(2022, 9, 1)
end = datetime(2022, 10, 31)

#df = pdr.get_data_yahoo_actions('FDX', start = start, end = end)
indice = 'FDX'
df = pdr.get_data_yahoo(indice, start = start, end = end)
df.head()

TypeError: string indices must be integers

#### Método PER

Relación precio_accion/utilidad_upa